## ML-第四版本-类别特征编码-Embeding特征
### 结果记录
+ embeding特征+lightgbm：训练集AUC均值：0.7393535126567923, 训练集合AUC方差: 0.006660455751244876
+ 类别特征+embeding+lightgbm: 训练集AUC均值：0.7515731176120227, 训练集合AUC方差: 0.005495153166907911
### 结论
+ embeding特征，从目前的做法来说，效果不好，embeding特征+类别特征woe的效果比不上单纯类别特征woe的效果

## 导包

In [14]:
import os
import re
import numpy as np
import pandas as pd
import datetime
import pickle as pkl

import matplotlib.pyplot as plt
import seaborn as sns


import scorpyo as sp
from null_importance import get_null_importance
from time_sequence_feats import get_time_base, get_sequence_statis, get_sequence_groupby_statis
import category_encoders as ce

from gensim.models import word2vec



import time
from sklearn.metrics import roc_auc_score as auc
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold

import warnings
warnings.filterwarnings("ignore")

pd.set_option('max_rows', 320, 'max_columns',100)

In [21]:
path_project = r'/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification'

# path dir
path_row_data = os.path.join(path_project, 'row_data')
path_new_data = os.path.join(path_project, 'new_data')
path_results  = os.path.join(path_project, 'results')
path_results_version  = os.path.join(path_results, 'version_2')

# path row_data
path_train = os.path.join(path_row_data, 'train.csv')
path_test  = os.path.join(path_row_data, 'evaluation_public.csv')
path_sample_submission = os.path.join(path_row_data, 'submit_example.csv')

## results
path_output_report = os.path.join(path_results, '01_原始数据探察_20221014.xlsx')

y_label = "is_risk"


# new_train_data
path_new_train_cate = os.path.join(path_new_data, 'train_cate.pkl')
path_new_test_cate  = os.path.join(path_new_data, 'test_cate.pkl')

path_new_train_time_sequence = os.path.join(path_new_data, 'train_time_sequence.pkl')
path_new_test_time_sequence  = os.path.join(path_new_data, 'test_time_sequence.pkl')

path_new_train_time_sequence_woe = os.path.join(path_new_data, 'train_time_sequence_woe.pkl')
path_new_test_time_sequence_woe  = os.path.join(path_new_data, 'test_time_sequence_woe.pkl')

path_new_train_cumsum = os.path.join(path_new_data, 'train_cumsum.pkl')
path_new_test_cumsum = os.path.join(path_new_data, 'test_cumsum.pkl')

path_new_train_cumsum_woe = os.path.join(path_new_data, 'train_cumsum_woe.pkl')
path_new_test_cumsum_woe = os.path.join(path_new_data, 'test_cumsum_woe.pkl')

path_new_train_embeding = os.path.join(path_new_data, 'train_embeding.pkl')
path_new_test_embeding = os.path.join(path_new_data, 'test_embeding.pkl')

In [16]:
df_row_train = pd.read_csv(path_train)
df_row_val  = pd.read_csv(path_test)

## 工具函数

In [17]:
def save_pkl(data, path_data):
    """将数据保存为pkl"""
    with open(path_data, 'wb') as f:
        pkl.dump(data, f)
        
def read_pkl(path_data):
    """读取pkl格式数据"""
    with open(path_data, 'rb') as f:
        data = pkl.load(f)
    return data

## Embeding特征

In [18]:
df = pd.concat([df_row_train, df_row_val]).reset_index(drop=True)
df = df.sort_values(by='op_datetime')

# 认证日期时间
df['op_datetime'] = pd.to_datetime(df['op_datetime'])
# 将数据分为每一天
df['op_days'] = df['op_datetime'].map(lambda x: x.strftime('%Y-%m-%d'))

### 词向量训练-word2vec

In [7]:
cate_feats = ['ip_transform', 'user_name', 'device_num_transform', 'department', 'browser_version', 'browser', 'os_type','os_version',
              'ip_type','http_status_code', 'op_city', 'log_system_transform', 'url']


In [8]:
for col in cate_feats:
    df_cols = df.groupby(['op_days'])[col].agg(lambda x: " ".join([str(i) for i in list(x)]))
    df_cols.to_csv(os.path.join(path_new_data, '{}.txt'.format(col)), index=False, header=False, sep='\t')

In [9]:
def train_embedding(path_corpus, path_save_models, path_save_txt, col):
    sentences = word2vec.Text8Corpus(path_corpus)  # 原始语料路径,已分词
    # 训练代码
    model = word2vec.Word2Vec(sentences, sg=1, vector_size=5, window=12, min_count=1,
                              hs=0,  workers=10, epochs=10)
    # save
    path_embedding_model = os.path.join(path_save_models, 'models_{}.model'.format(str(col)))
    path_embedding_vocab = os.path.join(path_save_txt, 'models_{}_embedding.txt'.format(str(col)))

    model.save(path_embedding_model)
    model.wv.save_word2vec_format(path_embedding_vocab)
    print('词向量训练完成：{}'.format(str(col)))

In [11]:
path_save_models = os.path.join(path_new_data, 'corpus_models')
path_save_txt = os.path.join(path_new_data, 'corpus_txt')

for path_tmp in [path_save_models, path_save_txt]:
    if not os.path.exists(path_tmp):
        os.makedirs(path_tmp)

for col in cate_feats:
    train_embedding(
        path_corpus = os.path.join(path_new_data, '{}.txt'.format(col)),
        path_save_models = path_save_models,
        path_save_txt = path_save_txt,
        col = col,
    )

词向量训练完成：ip_transform
词向量训练完成：user_name
词向量训练完成：device_num_transform
词向量训练完成：department
词向量训练完成：browser_version
词向量训练完成：browser
词向量训练完成：os_type
词向量训练完成：os_version
词向量训练完成：ip_type
词向量训练完成：http_status_code
词向量训练完成：op_city
词向量训练完成：log_system_transform
词向量训练完成：url


## Embeding特征合成

In [19]:
path_embeding = os.path.join(path_new_data, 'corpus_txt')

res_cols = []
for col in cate_feats:
    df_tmp = pd.read_csv(os.path.join(path_embeding, 'models_{}_embedding.txt'.format(col)), skiprows=1, header=None, sep=' ')
    df_tmp.columns = ['{}_{}'.format(col, i) for i in df_tmp.columns]
    df = pd.merge(left=df, right=df_tmp, how='left', left_on=col, right_on='{}_0'.format(col))
    res_cols.append('{}_0'.format(col))
    

remove_cols = ['ip_transform', 'user_name', 'device_num_transform', 'department', 'browser_version', 'browser', 'os_type','os_version',
              'ip_type','http_status_code', 'op_city', 'log_system_transform', 'url', 'op_datetime', 'op_month', 'op_days'] + res_cols

df = df.drop(columns=remove_cols)

In [22]:
df_train_embeding = df[df[y_label].notna()].reset_index(drop=True)
df_val_embeding= df[df[y_label].isna()].reset_index(drop=True)


save_pkl(df_train_embeding.drop(columns=[y_label]), path_new_train_embeding)
save_pkl(df_val_embeding.drop(columns=[y_label]), path_new_test_embeding)

## Modeling

### embeding特征+lightgbm

#### 获取建模使用的特征

In [24]:
remove_cols = ['id']
feats = df_train_embeding.columns
feats = feats.drop(remove_cols)
feats = feats.drop(y_label)

In [26]:
df_train = df_train_embeding.reset_index(drop=True)
df_test  = df_val_embeding.reset_index(drop=True)

#### lightgbm

In [27]:
# feats = feats_importance.sort_values('importance', ascending=False)[:50].name.values

In [28]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

res_sort_col = 'id'

In [29]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []


for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        # 数据集构造
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label]
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label]
                         )
        # 模型训练
        model = lgb.train(params, train, valid_sets=[train, val], 
                          num_boost_round=20000, early_stopping_rounds=100, verbose_eval=10)
        
        # 结果及得分获取
        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))

# 特征重要性获取
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

# 训练集得分效果
df_train['oof'] = oof
print('训练集AUC均值：{}, 训练集合AUC方差: {}'.format(np.mean(score), np.std(score)))

# 测试集合结果输出
score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values(res_sort_col).reset_index(drop=True)

if not os.path.exists(path_results_version):
    os.makedirs(path_results_version)

# 结果保存
sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_version,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.762596	valid_1's auc: 0.727419
[20]	training's auc: 0.769595	valid_1's auc: 0.725629
[30]	training's auc: 0.77421	valid_1's auc: 0.723639
[40]	training's auc: 0.778313	valid_1's auc: 0.723624
[50]	training's auc: 0.77972	valid_1's auc: 0.724474
[60]	training's auc: 0.781313	valid_1's auc: 0.724563
[70]	training's auc: 0.782694	valid_1's auc: 0.725097
[80]	training's auc: 0.786367	valid_1's auc: 0.725282
[90]	training's auc: 0.789432	valid_1's auc: 0.723382
[100]	training's auc: 0.791623	valid_1's auc: 0.722703
[110]	training's auc: 0.793599	valid_1's auc: 0.722192
Early stopping, best iteration is:
[10]	training's auc: 0.762596	valid_1's auc: 0.727419
----------- 1
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.761337	valid_1's auc: 0.736316
[20]	training's auc: 0.764872	valid_1's auc: 0.734796
[30]	training's auc: 0.768428	valid_1's auc: 0.737245
[40

,name,importance
46,http_status_code_2,29792.899253
3,ip_transform_4,8922.023632
47,http_status_code_3,8602.106845
2,ip_transform_3,4290.868230
1,ip_transform_2,3522.781572
4,ip_transform_5,3472.915336
15,department_1,2769.047775
5,user_name_1,1982.249963
8,user_name_4,1509.059508
0,ip_transform_1,1435.076991


训练集AUC均值：0.7393535126567923, 训练集合AUC方差: 0.006660455751244876


### 类别特征+embeding+lightgbm

#### 类别特征+交叉累计特征

In [31]:
df_train_cate = read_pkl(path_new_train_cate)
df_val_cate = read_pkl(path_new_test_cate)

df_train_embeding = read_pkl(path_new_train_embeding)
df_val_embeding = read_pkl(path_new_test_embeding)

df_raw_train = pd.read_csv(path_train)
df_raw_val  = pd.read_csv(path_test)

In [32]:
df_row_train = pd.merge(left=df_train_cate, right=df_train_embeding, on='id')
df_row_val = pd.merge(left=df_val_cate, right=df_val_embeding, on='id')

df_row_train = pd.merge(left=df_row_train, right=df_raw_train[['id', y_label]], on='id')
df_row_val = pd.merge(left=df_row_val, right=df_raw_val[['id']], on='id')

In [33]:
df_row_train.head()

,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,ip_type,http_status_code,op_city,log_system_transform,url,ip_transform_1,ip_transform_2,ip_transform_3,ip_transform_4,ip_transform_5,user_name_1,user_name_2,user_name_3,user_name_4,user_name_5,device_num_transform_1,device_num_transform_2,device_num_transform_3,device_num_transform_4,device_num_transform_5,department_1,department_2,department_3,department_4,department_5,browser_version_1,browser_version_2,browser_version_3,browser_version_4,browser_version_5,browser_1,browser_2,browser_3,browser_4,browser_5,os_type_1,os_type_2,os_type_3,os_type_4,os_type_5,os_version_1,os_version_2,os_version_3,os_version_4,os_version_5,ip_type_1,ip_type_2,ip_type_3,ip_type_4,ip_type_5,http_status_code_1,http_status_code_2,http_status_code_3,http_status_code_4,http_status_code_5,op_city_1,op_city_2,op_city_3,op_city_4,op_city_5,log_system_transform_1,log_system_transform_2,log_system_transform_3,log_system_transform_4,log_system_transform_5,url_1,url_2,url_3,url_4,url_5,is_risk
0,0,0.399244,-0.041851,0.123852,0.355653,0.014496,0.014496,0.009044,0.012388,-0.011044,-0.298867,-0.100602,-0.246014,-0.246014,0.351102,0.662821,0.136250,-1.463449,0.109411,-0.238206,-0.112968,0.385698,-0.693542,-0.395487,0.337768,-0.222236,0.985193,-0.984660,-0.199364,-0.264394,0.343111,0.333835,1.060045,0.304785,0.398715,0.412455,0.821953,-0.308019,-0.312888,-0.258411,-0.764207,0.387000,0.265972,-0.007717,-0.040867,0.155221,0.159225,-0.079858,0.754709,0.482463,-0.009534,0.425888,0.791858,0.259018,0.177898,-0.120132,-0.513428,-0.17467,0.296901,0.205209,-0.554731,-0.563044,0.313156,0.769898,-0.003393,-0.135347,0.100012,0.717724,-0.495645,0.127078,0.504821,0.621537,0.007778,-0.246429,-0.278626,0.566518,0.582656,-0.123729,-0.093712,0
1,1,0.090044,0.035442,-0.076528,0.012419,0.060103,-0.009554,0.009044,-0.021990,-0.011044,-0.298867,-0.069165,-0.150937,-0.150937,-0.010517,-0.236595,1.005191,-1.392806,0.030540,0.473811,-1.028081,-0.141099,-0.677721,-0.261522,0.603597,-1.067501,1.029901,-0.400787,-0.060752,-0.173541,0.320625,0.392769,1.087662,0.149204,0.088469,0.265654,0.979246,-0.383104,-0.094422,0.059363,-0.634294,0.328273,0.470747,0.179674,-0.040867,0.155221,0.159225,-0.079858,0.754709,0.578314,0.116210,0.386729,0.860731,-0.148265,0.177898,-0.120132,-0.513428,-0.17467,0.296901,0.205209,-0.554731,-0.563044,0.313156,0.769898,-0.065779,-0.065941,0.106161,0.741891,-0.477608,0.145657,0.534680,0.523185,0.078129,-0.308373,-0.298457,0.549703,0.546102,-0.142218,-0.140829,0
2,2,0.142991,-0.041851,-0.098600,0.092439,-0.059518,-0.068291,0.009044,0.012388,-0.011044,-0.298867,-0.100602,0.530533,-0.206373,0.881575,0.871942,0.377879,-1.090439,0.105910,0.123588,-0.069507,0.040772,-0.923039,-0.341436,0.359016,-0.097269,1.228916,-0.624206,-0.155549,-0.264394,0.343111,0.333835,1.060045,0.304785,-0.055066,0.758656,0.756397,-0.201964,-0.086228,-0.103458,-0.675540,0.021527,0.537719,0.324517,-0.040867,0.155221,0.159225,-0.079858,0.754709,0.482463,-0.009534,0.425888,0.791858,0.259018,0.177898,-0.120132,-0.513428,-0.17467,0.296901,0.205209,-0.554731,-0.563044,0.313156,0.769898,-0.003393,-0.135347,0.100012,0.717724,-0.495645,0.272324,0.801401,0.477765,-0.207261,-0.093121,-0.219113,0.473965,0.617964,-0.297655,-0.096366,0
3,3,-0.011160,-0.041851,-0.255592,-0.085693,0.041217,0.041217,0.009044,0.012388,-0.011044,-0.298867,0.034066,-0.284901,-0.284901,-0.042532,1.034272,0.496798,-1.318175,0.197559,-0.618682,-0.032847,0.514913,-0.790935,-0.321877,0.378910,0.352033,1.057229,-0.916724,-0.358204,-0.264394,0.343111,0.333835,1.060045,0.304785,0.258411,0.474229,0.870716,-0.160924,-0.083712,0.052082,-0.621474,0.354803,0.474141,0.237456,-0.040867,0.155221,0.159225,-0.079858,0.754709,0.482463,-0.009534,0.425888,0.791858,0.259018,0.177898,-0.120132,-0.513428,-0.17467,0.296901,0.205209,-0.554731,-0.563044,0.313156,0.769898,0.083323,-0.179963,0.140210,0.713114,-0.469756,-0.041631,0.556606,0.498043,0.079124,-0.384638,

#### 获取建模使用的特征

In [42]:
remove_cols = ['id']
feats = df_row_train.columns
feats = feats.drop(remove_cols)
feats = feats.drop(y_label)

In [43]:
df_train = df_row_train.reset_index(drop=True)
df_test  = df_row_val.reset_index(drop=True)

#### lightgbm

In [44]:
# feats = feats_importance.sort_values('importance', ascending=False)[:50].name.values

In [45]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

res_sort_col = 'id'

In [46]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []


for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        # 数据集构造
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label]
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label]
                         )
        # 模型训练
        model = lgb.train(params, train, valid_sets=[train, val], 
                          num_boost_round=20000, early_stopping_rounds=100, verbose_eval=10)
        
        # 结果及得分获取
        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))

# 特征重要性获取
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

# 训练集得分效果
df_train['oof'] = oof
print('训练集AUC均值：{}, 训练集合AUC方差: {}'.format(np.mean(score), np.std(score)))

# 测试集合结果输出
score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values(res_sort_col).reset_index(drop=True)

if not os.path.exists(path_results_version):
    os.makedirs(path_results_version)

# 结果保存
sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_version,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.769231	valid_1's auc: 0.753631
[20]	training's auc: 0.77387	valid_1's auc: 0.751908
[30]	training's auc: 0.776123	valid_1's auc: 0.752333
[40]	training's auc: 0.777015	valid_1's auc: 0.752354
[50]	training's auc: 0.778465	valid_1's auc: 0.75151
[60]	training's auc: 0.779632	valid_1's auc: 0.75076
[70]	training's auc: 0.781393	valid_1's auc: 0.75034
[80]	training's auc: 0.783512	valid_1's auc: 0.75047
[90]	training's auc: 0.786404	valid_1's auc: 0.748084
[100]	training's auc: 0.788632	valid_1's auc: 0.747224
Early stopping, best iteration is:
[2]	training's auc: 0.761701	valid_1's auc: 0.758415
----------- 1
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.774534	valid_1's auc: 0.740603
[20]	training's auc: 0.778593	valid_1's auc: 0.738381
[30]	training's auc: 0.78224	valid_1's auc: 0.739221
[40]	training's auc: 0.784396	valid_1's auc: 0.737462
[50]	trai

,name,importance
2,ip_transform,10937.080440
9,http_status_code,9862.422724
59,http_status_code_2,6089.804824
60,http_status_code_3,2306.879980
16,ip_transform_4,706.616425
8,ip_type,544.837718
15,ip_transform_3,397.289082
17,ip_transform_5,341.762145
18,user_name_1,251.979293
14,ip_transform_2,251.243784


训练集AUC均值：0.7515731176120227, 训练集合AUC方差: 0.005495153166907911
